In [ ]:
import numpy as np
import pandas as pd

from lightgbm import LGBMRegressor

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

In [ ]:
columns = data.drop(['id', 'claim'], axis=1).columns

In [ ]:
missing_data_indicator = data[columns].isna()
missing_test_indicator = test[columns].isna()
data_imputed = data.copy()
test_imputed = test.copy()
for column in columns:
    print(f'Imputing column {column}..')
    X_train = data[missing_data_indicator[column] == False].drop([column, 'id', 'claim'], axis=1)
    y_train = data[missing_data_indicator[column] == False][column]
    X_missing = data[missing_data_indicator[column] == True].drop([column, 'id', 'claim'], axis=1)
    X_test_missing = test[missing_test_indicator[column] == True].drop([column, 'id'], axis=1)
    model = LGBMRegressor()
    print('Fitting model..')
    model.fit(X_train, y_train)
    print(f'Predicting missing {column} values in the train set..')
    y_missing = model.predict(X_missing)
    data_imputed.loc[missing_data_indicator[column] == True, column] = y_missing
    print(f'Predicting missing {column} values in the test set..')
    y_test_missing = model.predict(X_test_missing)
    test_imputed.loc[missing_test_indicator[column] == True, column] = y_test_missing
    print(f'Column {column} imputed!')
    print()
print('Done!')

In [ ]:
missing_data_indicator.to_csv('missing_data_indicator.csv', index=False)
missing_test_indicator.to_csv('missing_test_indicator.csv', index=False)
data_imputed.to_csv('train_imputed.csv', index=False)
test_imputed.to_csv('test_imputed.csv', index=False)